# Topic modelling of news headlines for prediction of news category with 6 topics

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier and assume 6 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import wordcloud

import pyLDAvis.lda_model

## Load train set

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Extract features from 'title' 

In [7]:
train[train['title_cleaned'].isnull()]

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned
39381,2023-02-16 15:30:15,Champions League,Liveticker und Highlightvideos,Missing,NaN,ZDF heute,Champions League - Liveticker und Highlightvid...,NaN,Champion Liveticker Highlightvideos
66356,2023-09-21 00:30:05,Champions League,Die wichtigsten Spiele im Video.,Missing,NaN,ZDF heute,Champions League - Die wichtigsten Spiele im V...,NaN,wichtig Spiel Video


In [8]:
# TODO workaround for missing title_cleaned -> TODO Solve this in NLP preprocessing!
train['title_cleaned'] = train['title_cleaned'].fillna(train['title'].apply(lambda x: x.lower()))

In [9]:
train[train['title_cleaned'].isnull()]

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)

In [12]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [13]:
dtm

<67513x21412 sparse matrix of type '<class 'numpy.int64'>'
	with 330205 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.decomposition import LatentDirichletAllocation

In [15]:
lda = LatentDirichletAllocation(n_components=6, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=6, random_state=42)

## Analyse extracted features

In [16]:
len(cv.get_feature_names_out())

21412

In [17]:
cv.get_feature_names_out()

array(['000', '034', '039', ..., 'войны', 'российские', 'украинцы'],
      dtype=object)

In [18]:
len(lda.components_)

6

In [19]:
lda.components_

array([[ 1.33150739, 16.87201671,  6.54814522, ...,  0.16666854,
         0.16666854,  0.16667097],
       [ 2.91656852,  5.13646195,  0.16790719, ...,  2.16665603,
         2.16665603,  0.16667174],
       [12.2518729 ,  8.9610429 , 29.85138169, ...,  0.16666872,
         0.16666872,  0.16667137],
       [ 0.16666822, 33.27981301,  4.70423329, ...,  0.16666891,
         0.16666891,  0.16667181],
       [ 0.16671478, 19.02249721, 15.56146644, ...,  0.16666894,
         0.16666894,  2.16664244],
       [ 0.16666819,  6.72816821,  0.16686616, ...,  0.16666886,
         0.16666886,  0.16667169]])

In [20]:
len(lda.components_[0])

21412

### Show most important words of first extracted topic

In [21]:
first_topic = lda.components_[0]

In [22]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([ 3800, 19585, 20969, ..., 15087, 10150, 18325], dtype=int64)

In [23]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [24]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

liveblog
selenskyj
kiew
angriff
lage
putin
russland
russisch
krieg
ukraine


### Show most important words of first extracted topic

In [25]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['moskau', 'chef', 'stimme', 'invasion', 'ukrainisch', 'liveblog', 'selenskyj', 'kiew', 'angriff', 'lage', 'putin', 'russland', 'russisch', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #1
['milliarde', 'panzer', 'kritik', 'faeser', 'land', 'spd', 'fdp', 'wahrheit', 'lindner', 'habeck', 'streit', 'eu', 'regierung', 'euro', 'ampel']


Die TOP-15 Wörter für das Thema #2
['million', 'eu', 'welt', 'taiwan', 'groß', 'analyse', 'marktbericht', 'wm', 'biden', 'usa', 'berlin', 'scholz', 'us', 'deutschland', 'china']


Die TOP-15 Wörter für das Thema #3
['brauchen', 'partei', 'groß', 'deutschland', 'g7', 'afd', 'türkei', 'klima', 'frankreich', 'gipfel', 'russland', 'wahl', 'israel', 'putin', 'deutsch']


Die TOP-15 Wörter für das Thema #4
['europa', 'protest', 'mann', 'gas', 'union', 'sanktion', 'cdu', 'gastbeitrag', 'grüne', 'bundesliga', 'tote', 'frau', 'iran', 'bayern', 'eu']


Die TOP-15 Wörter für das Thema #5
['erwarten', 'steigen', 'mensch', 'ne

### Get topics with highest probability for news items in train data

In [26]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 6)

In [27]:
topic_results[0].round(5)

array([0.02086, 0.25029, 0.0209 , 0.29752, 0.24661, 0.16382])

In [28]:
topic_results[0].argmax()

3

In [29]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [30]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...,3
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...,0
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...,1
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...,0
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...,1


## Visualize model

In [31]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0     -83.283760  44.359619       1        1  18.793372
2     -17.855074  91.507027       2        1  17.134110
1     -58.269043 -32.293758       3        1  16.373832
5      22.107683 -32.159653       4        1  16.102300
3     -18.216829  23.200260       5        1  15.955264
4      47.030670  44.166130       6        1  15.641122, topic_info=            Term         Freq        Total Category  logprob  loglift
18325    ukraine  4537.000000  4537.000000  Default  30.0000  30.0000
10150      krieg  2629.000000  2629.000000  Default  29.0000  29.0000
15087   russisch  1381.000000  1381.000000  Default  28.0000  28.0000
3583      corona  1141.000000  1141.000000  Default  27.0000  27.0000
15091   russland  1762.000000  1762.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
9854   kommentar   142.170196   309.612713   Topic6  -5.9019   1.0770
10650      leben   145.131934   358.676154   Topic6  -5.8812   0.9505
7647      habeck   157.369173   712.800854   Topic6  -5.8003   0.3447
6369     fußball   143.523851   337.967882   Topic6  -5.8924   0.9988
2314      berlin   154.548740   982.513856   Topic6  -5.8184   0.0057

[362 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
333        2  0.980582      abtreibung
418        2  0.172287             afd
418        3  0.168841             afd
418        5  0.520307             afd
418        6  0.137830             afd
...      ...       ...             ...
20777      3  0.075062          zahlen
21262      3  0.896901      österreich
21262      5  0.104291      österreich
21275      1  0.993195       überblick
21368      6  0.989398  überschwemmung

[650 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 6, 4, 5])

## Save train set with topics and dump model

In [32]:
#Save result to csv
train.to_csv('train_lda_topics_6.csv')

In [33]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv.jl')

['models/cv.jl']

In [34]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_6_topics.jl')

['models/lda_model_6_topics.jl']